In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter
from skimage.feature import peak_local_max
import tifffile as tiff
import os
import matplotlib.pyplot as plt

# Function to create a sphere
def create_sphere(center, radius, shape):
    z, y, x = np.indices(shape)
    distance = np.sqrt((z - center[0])**2 + (y - center[1])**2 + (x - center[2])**2)
    return distance <= radius

# Function to create a mask for a single 3D image
def generate_mask(image_3d, sigma=(1, 1, 1), threshold_percentile=99, spot_radius=2):
    # Apply Gaussian filter
    smoothed_image = gaussian_filter(image_3d, sigma=sigma)

    # Thresholding to find bright spots
    threshold = np.percentile(smoothed_image, threshold_percentile)
    binary_image = smoothed_image > threshold

    # Find local maxima for spot centers
    coordinates = peak_local_max(smoothed_image, min_distance=2, threshold_abs=threshold)

    # Create an empty mask
    mask = np.zeros_like(image_3d, dtype=np.uint8)

    # Add spheres to the mask at detected spot locations
    for coord in coordinates:
        z, y, x = coord
        sphere_mask = create_sphere((z, y, x), spot_radius, image_3d.shape)
        mask[sphere_mask] = 1

    return mask

# Load the 5D image (tzcyx)
image_path = "2024-11-07_MC256_slide6_Capture_8_cropped.tif"  # Replace with your image file path
image = tiff.imread(image_path)  # Shape: (time, z, channel, y, x)

# Output file
output_file = "spotmask.tif"  # Replace with your desired output path

# Ensure the image is in tzcyx format and process channel 1 (index 0)
assert image.ndim == 5, "Input image must be a 5D stack (tzcyx)."
channel_index = 0  # Index for channel 1
time_points, z_slices, _, height, width = image.shape
processed_masks = np.zeros((time_points, z_slices, height, width), dtype=np.uint8)  # tz, y, x

# Process each time point
for t in range(time_points):
    print(f"Processing time point {t + 1}/{time_points}...")
    image_3d = image[t, :, channel_index, :, :]  # Extract zyx stack for channel 1
    mask = generate_mask(image_3d, sigma=(1, 1, 1), threshold_percentile=99, spot_radius=2)
    processed_masks[t] = mask  # Save the mask

# Save the output mask as a 5D TIFF stack
tiff.imwrite(output_file, processed_masks, dtype=np.uint8)
print(f"Processing complete. Masks saved to: {output_file}")


Processing time point 1/127...


In [4]:
import cupy as cp
from cupyx.scipy.ndimage import gaussian_filter
from skimage.feature import peak_local_max
import tifffile as tiff
import numpy as np  # Still used for some CPU-based operations

# Function to create a sphere
def create_sphere(center, radius, shape):
    z, y, x = cp.indices(shape)
    distance = cp.sqrt((z - center[0])**2 + (y - center[1])**2 + (x - center[2])**2)
    return distance <= radius

# Function to create a mask for a single 3D image
def generate_mask(image_3d, sigma=(1, 1, 1), threshold_percentile=99.5, spot_radius=2):
    # Transfer image to GPU
    image_3d_gpu = cp.asarray(image_3d)

    # Apply Gaussian filter on GPU
    smoothed_image = gaussian_filter(image_3d_gpu, sigma=sigma)

    # Thresholding to find bright spots (ensure threshold is NumPy scalar)
    threshold = cp.percentile(smoothed_image, threshold_percentile).item()
    smoothed_image_np = cp.asnumpy(smoothed_image)

    # Find local maxima for spot centers
    coordinates = peak_local_max(
        smoothed_image_np, min_distance=2, threshold_abs=threshold
    )
    
    # Count the number of spots
    num_spots = len(coordinates)
    print(f"Number of spots detected: {num_spots}")

    # Create an empty mask on GPU
    mask = cp.zeros_like(image_3d_gpu, dtype=cp.uint8)

    # Add spheres to the mask at detected spot locations
    for coord in coordinates:
        z, y, x = coord
        sphere_mask = create_sphere((z, y, x), spot_radius, image_3d_gpu.shape)
        mask[sphere_mask] = 1

    # Transfer mask back to CPU
    return cp.asnumpy(mask), num_spots

# Load the 5D image (tzcyx)
image_path = "2024-11-07_MC256_slide6_Capture_8_cropped.tif"  # Replace with your image file path
image = tiff.imread(image_path)  # Shape: (time, z, channel, y, x)

threshold = 99.7
sigma = 2
spot_radius = 3
# Output file
output_file = f"spotmask_every_10th_threshold{threshold}_sigma_{sigma}_spotradius{spot_radius}.tif"  # Replace with your desired output path

# Ensure the image is in tzcyx format and process channel 1 (index 0)
assert image.ndim == 5, "Input image must be a 5D stack (tzcyx)."
channel_index = 0  # Index for channel 1
time_points, z_slices, _, height, width = image.shape
processed_masks = np.zeros((time_points, z_slices, height, width), dtype=np.uint8)  # tz, y, x

# Process every 10th time point
total_spots = 0
for t in range(0, time_points, 1):
    print(f"Processing time point {t + 1}/{time_points}...")
    image_3d = image[t, :, channel_index, :, :]  # Extract zyx stack for channel 1
    mask, num_spots = generate_mask(image_3d, sigma=(sigma, sigma, sigma), threshold_percentile=threshold, spot_radius=spot_radius)
    processed_masks[t] = mask  # Save the mask for the processed time point
    total_spots += num_spots
    print(f"Frame {t}: Detected {num_spots} spots")

# Print the total number of spots across all frames
print(f"Total spots detected across all processed frames: {total_spots}")

# Save the output mask as a 5D TIFF stack
# Function to save the hyperstack TIFF
def save_hyperstack_tiff(segmented_movie, output_path):
    tiff.imwrite(
        output_path,
        segmented_movie,
        imagej=True,
        metadata={'axes': 'TZYX'},
        resolution=(1/0.1625, 1/0.1625),
    )
    print(f"Saved segmented movie as ImageJ hyperstack to {output_path}")


save_hyperstack_tiff(processed_masks, output_file)
print(f"Processing complete. Masks for every 10th frame saved to: {output_file}")


Processing time point 1/127...
Number of spots detected: 291
Frame 0: Detected 291 spots
Processing time point 2/127...
Number of spots detected: 228
Frame 1: Detected 228 spots
Processing time point 3/127...
Number of spots detected: 233
Frame 2: Detected 233 spots
Processing time point 4/127...
Number of spots detected: 228
Frame 3: Detected 228 spots
Processing time point 5/127...
Number of spots detected: 234
Frame 4: Detected 234 spots
Processing time point 6/127...
Number of spots detected: 248
Frame 5: Detected 248 spots
Processing time point 7/127...
Number of spots detected: 232
Frame 6: Detected 232 spots
Processing time point 8/127...
Number of spots detected: 249
Frame 7: Detected 249 spots
Processing time point 9/127...
Number of spots detected: 265
Frame 8: Detected 265 spots
Processing time point 10/127...
Number of spots detected: 250
Frame 9: Detected 250 spots
Processing time point 11/127...
Number of spots detected: 235
Frame 10: Detected 235 spots
Processing time po

In [ ]:
import cupy as cp
from cupyx.scipy.ndimage import gaussian_filter
from skimage.feature import peak_local_max
import tifffile as tiff
import numpy as np

# Function to create a sphere
def create_sphere(center, radius, shape):
    z, y, x = cp.indices(shape)
    distance = cp.sqrt((z - center[0])**2 + (y - center[1])**2 + (x - center[2])**2)
    return distance <= radius

# Function to create a mask for a batch of 3D images
def generate_mask_batch(images_3d, sigma=(1, 1, 1), threshold_percentile=99, spot_radius=2):
    # Transfer images to GPU
    images_gpu = cp.asarray(images_3d)

    # Apply Gaussian filter to all images in the batch
    smoothed_images = cp.array([gaussian_filter(img, sigma=sigma) for img in images_gpu])

    # Process each image individually
    masks = []
    for smoothed_image in smoothed_images:
        # Thresholding to find bright spots
        threshold = cp.percentile(smoothed_image, threshold_percentile).item()
        smoothed_image_np = cp.asnumpy(smoothed_image)

        # Find local maxima
        coordinates = peak_local_max(
            smoothed_image_np, min_distance=2, threshold_abs=threshold
        )

        # Create an empty mask
        mask = cp.zeros_like(smoothed_image, dtype=cp.uint8)

        # Add spheres to the mask at detected spot locations
        for coord in coordinates:
            z, y, x = coord
            sphere_mask = create_sphere((z, y, x), spot_radius, smoothed_image.shape)
            mask[sphere_mask] = 1

        # Save the mask
        masks.append(cp.asnumpy(mask))

    return masks

# Function to save the hyperstack TIFF
def save_hyperstack_tiff(segmented_movie, output_path):
    tiff.imwrite(
        output_path,
        segmented_movie,
        imagej=True,
        metadata={'axes': 'TZYX'},
        resolution=(1/0.1625, 1/0.1625),
    )
    print(f"Saved segmented movie as ImageJ hyperstack to {output_path}")

# Load the 5D image (tzcyx)
image_path = "2024-11-07_MC256_slide6_Capture_8_cropped.tif"
image = tiff.imread(image_path)  # Shape: (time, z, channel, y, x)

# Output file
output_file = "spotmask_hyperstack.tif"

# Ensure the image is in tzcyx format and process channel 1 (index 0)
assert image.ndim == 5, "Input image must be a 5D stack (tzcyx)."
channel_index = 0  # Index for channel 1
time_points, z_slices, _, height, width = image.shape
processed_masks = np.zeros((time_points, z_slices, height, width), dtype=np.uint8)

# Set batch size
batch_size = 4  # Number of time points to process in parallel

# Process in batches
for batch_start in range(0, time_points, batch_size):
    batch_end = min(batch_start + batch_size, time_points)
    print(f"Processing time points {batch_start + 1} to {batch_end}...")
    
    # Extract batch
    batch_images = image[batch_start:batch_end, :, channel_index, :, :]  # Shape: (batch, z, y, x)
    
    # Generate masks for the batch
    batch_masks = generate_mask_batch(batch_images, sigma=(1, 1, 1), threshold_percentile=99, spot_radius=2)

    # Save the masks back into the full array
    for i, mask in enumerate(batch_masks):
        processed_masks[batch_start + i] = mask

# Save the output mask as a 5D TIFF hyperstack
save_hyperstack_tiff(processed_masks, output_file)


Processing time points 1 to 4...
Processing time points 5 to 8...
